In [1]:
import pandas as pd
import os
absolutepath = os.path.abspath(os.getcwd())
fileDirectory = os.path.dirname(absolutepath)
parentDirectory = os.path.dirname(fileDirectory)
dataprocDirectory = os.path.join(fileDirectory, 'data/proc')
datarawDirectory = os.path.join(fileDirectory, 'data/raw')

/cluster/home/jiycai/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
import psycopg2
import logging

In [3]:
def return_candiate_pool_affiliation_history():
    # each row contains a authid X afid spell
    sql_text = f"""
    select scopus_authors_aff_year_filled_chinese.authid,scopus_authors_aff_year_filled_chinese.afid,
    subfield_most_frequent,
    subfield_most_frequent_two_digit,
    scopus_affiliation.jttp_school,
    year
    from scopus_authors_aff_year_filled_chinese
    left join scopus_affiliation on 
    scopus_authors_aff_year_filled_chinese.afid = scopus_affiliation.afid
    left join all_authors_in_restriction_jttp on 
    scopus_authors_aff_year_filled_chinese.authid = all_authors_in_restriction_jttp.authid
    where all_authors_in_restriction_jttp.restriction = 1;
    """
    
    con = psycopg2.connect(host='id-hdb-psgr-cp7.ethz.ch',  
                      dbname='led', 
                      user='jiycai')

    cur = con.cursor()
    cur.execute(sql_text)
    rows = cur.fetchall()
    con.close()
    columns = ['authid','afid',
               'subfield_most_frequent',
               'subfield_most_frequent_two_digit',
               'jttp_school',
               'year']
    data = pd.DataFrame(rows,columns=columns)
    return data

In [4]:
data = return_candiate_pool_affiliation_history()

In [5]:
data.shape

(10440163, 6)

# check for gaps in affiliation size

In [6]:
data['max_year'] = data[['afid','authid','year']].groupby(['afid','authid']).transform('max')[['year']]
data['min_year'] = data[['afid','authid','year']].groupby(['afid','authid']).transform('min')[['year']]
data['spell_length'] = data.max_year - data.min_year + 1 

In [7]:
len(data[data.spell_length>1].afid.unique())

61529

In [8]:
groupby_var = ['afid','subfield_most_frequent_two_digit']
groupby_var_outcome = groupby_var + ['authid']
data_affXafield_size = data[groupby_var_outcome].groupby(groupby_var).nunique().reset_index()

In [9]:
data_affXafield_size.columns = ['afid','subfield_most_frequent_two_digit','affXfield_size']

In [10]:
data_affXafield_size = data_affXafield_size.sort_values('affXfield_size')

In [11]:
data = data.merge(data_affXafield_size, how = 'left', on = groupby_var)

In [12]:
groupby_var = ['afid']
groupby_var_outcome = groupby_var + ['authid']
data_aff_size = data[groupby_var_outcome].groupby(groupby_var).nunique().reset_index()

In [13]:
data_aff_size = data_aff_size.sort_values('authid')

In [14]:
data_aff_size.columns = ['afid','afid_size']

In [15]:
data = data.merge(data_aff_size, how = 'left', on = 'afid')

In [16]:
data['afid_masked'] = data['afid']
data.loc[data.afid_size<100,'afid_masked'] = 'masked'

In [17]:
data.to_csv(os.path.join(dataprocDirectory, 'author_pool_affiliation_history.csv'),index=False)

# create affiliation list for each year

In [18]:
groupby_var = ['authid','year']
groupby_var_outcome = groupby_var + ['afid_masked']
data_vec = data[groupby_var_outcome].groupby(groupby_var).agg(['unique','nunique']).reset_index()
data_vec.columns = ['_'.join(col) for col in data_vec.columns]

In [19]:
data_vec.to_csv(os.path.join(dataprocDirectory, 'author_pool_affiliation_vectors_list_form.csv'),index=False)